With the above code, we have to initialize the class evry epoch, it fastens the things
Learn abou how pytorch hadles the bath loader and see if it has to do with the _length__
We can trigger the length with len(users)*(num_neg_instaces+1) and then use idx in __getitem_

idx
int(idx/num_negative_instaces)

this will work only if the list is sorted, which it isn't.
we can try to see if the idx position only starts one time at 0 and then call the method to generate data. we have to ensure that the paraleliation respects this wchich I doubt. 

In [ ]:
from random import sample, seed

def _create_negative_sample(train_data, test_data, user_column:str, item_column:str, num_samples:int=None):
  # Set a random seed for reproducibility if needed
  seed(42)

  # Create set of all items
  total_items = set(test_data[item_column].unique()) | set(train_data[item_column].unique())
  
  # Creating a dictionary to keep track of items each user has interacted with in train_data
  items_per_user_train = train_data.groupby(user_column)[item_column].apply(set).to_dict()
  items_per_user_test = test_data.groupby(user_column)[item_column].apply(set).to_dict()
  
  negative_samples = []

  for index, row in test_data.iterrows():
    user = row[user_column]
    item = row[item_column]
    
    # Get interacted items from both train and test data, or an empty set if the user doesn't exist
    items_train = items_per_user_train.get(user, set())
    items_test = items_per_user_test.get(user, set())
    
    interacted_items = items_train | items_test
    non_interacted_items = list(total_items - interacted_items)
    
    if num_samples and len(non_interacted_items) >= num_samples:
      pool_of_items = sample(non_interacted_items, num_samples)
    else: 
      pool_of_items = non_interacted_items
    
    
    negative_samples.append([(user, item)] + pool_of_items)

  return negative_samples


In [ ]:
from random import sample
def create_negative_sample(train_data, test_data, user_column:str, item_column:str, num_samples:int=None):
  train_data_grouped = train_data.groupby([user_column, item_column]).count()
  total_items = set(test_data[item_column].unique()) | set(train_data[item_column].unique())
  negative_samples = []

  for row in test_data.iterrows():
    user = int(row[1][user_column])
    item = int(row[1][item_column])
    items_train = set(train_data_grouped.loc[user].index)
    items_test = set(test_data[test_data[user_column] == user][item_column])
    iteracted_items = items_train | items_test
    non_interacted_items = list(total_items - iteracted_items)
    if num_samples:
      pool_of_items = sample(non_interacted_items, 99)
    else: 
      pool_of_items = non_interacted_items
    negative_samples.append(([(user, item)] + pool_of_items))

  return negative_samples

create_negative_sample(train_data, test_data, 'user', 'item')

In [ ]:
from random import sample
train_data = df_frappe.train_ratings
test_data = df_frappe.test_ratings

train_data_grouped = train_data.groupby(['user', 'item']).count()
total_items = set(test_data['item'].unique()) | set(train_data['item'].unique())
negative_samples = []
for user in test_data.user.unique():
  items_train = set(train_data_grouped.loc[0].index)
  items_test = set(test_data[test_data['user'] == user]['item'])
  iteracted_items = items_train | items_test
  non_interacted_items = list(total_items - iteracted_items)
  pool_of_items = sample(non_interacted_items, 99)
  negative_samples.append(([(user, item)] + random_elements))
  
  break

In [ ]:
train_data_grouped = train_data.groupby(['user', 'item']).count()
total_items = set(test_data['item'].unique()) | set(train_data['item'].unique())
negative_samples = []

for row in test_data.iterrows():
  user = int(row[1]['user'])
  item = int(row[1]['item'])
  items_train = set(train_data_grouped.loc[0].index)
  items_test = set(test_data[test_data['user'] == user]['item'])
  iteracted_items = items_train | items_test
  non_interacted_items = list(total_items - iteracted_items)
  pool_of_items = sample(non_interacted_items, 99)
  negative_samples.append(([(user, item)] + pool_of_items))

